In [1]:
import pyspark.sql.functions as func
from pyspark.sql import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import rand
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
spark = SparkSession.builder \
    .appName("Restaurant Recommendation") \
    .getOrCreate()

spark.conf.set("spark.sql.legacy.style", "false")

In [3]:
cwd = os.getcwd()
business_df = spark.read.json('C:/Users/Haley/OneDrive - Georgia Institute of Technology/ISyE 6740/Project/yelp_dataset/yelp_academic_dataset_business.json')
reviews_df = spark.read.json('C:/Users/Haley/OneDrive - Georgia Institute of Technology/ISyE 6740/Project/yelp_dataset/yelp_academic_dataset_review.json')
users_df = spark.read.json('C:/Users/Haley/OneDrive - Georgia Institute of Technology/ISyE 6740/Project/yelp_dataset/yelp_academic_dataset_user.json')

In [ ]:
business_df.columns

In [ ]:
business_df.show()

In [ ]:
users_df.columns

In [ ]:
reviews_df.columns

In [5]:
#Generic numeric user_id and business_id to use later for modeling functions
user_window = Window.orderBy("user_id")
business_window = Window.orderBy("business_id")

users_df = users_df.withColumn("user_number", func.row_number().over(user_window)) #May need to do this only for PA
business_df = business_df.withColumn("business_number", func.row_number().over(business_window))

#Create temp views for SQL
business_df.createOrReplaceTempView("business")
reviews_df.createOrReplaceTempView("reviews")
users_df.createOrReplaceTempView("users")

In [6]:
users_df.select(func.max(users_df['user_number'])).collect()[0][0]

1987897

In [7]:
#Data Filtering
state = 'PA'

#Restaurants
restaurant_sql = f'''
        SELECT 
            b.*
        FROM 
            business b
        WHERE 
            b.is_open = 1 AND LOWER(b.categories) LIKE '%restaurant%' AND b.state = '{state}'
        ;
    '''

restaurants = spark.sql(restaurant_sql)
restaurants.createOrReplaceTempView("restaurants")


#Reviews for Restaurants
restaurant_reviews_sql = f'''
        SELECT 
            rv.*, r.city, r.state
        FROM 
            reviews rv
        INNER JOIN
            restaurants r
        ON rv.business_id = r.business_id
        ;
    '''

restaurant_reviews = spark.sql(restaurant_reviews_sql)
restaurant_reviews.createOrReplaceTempView("restaurant_reviews")

#Restaurant review volume by state
restaurant_review_volume_sql = f'''
        SELECT 
           rv.state, count(*) as review_count
        FROM 
            restaurant_reviews rv
        GROUP BY rv.state ORDER BY review_count DESC
        ;
    '''

restaurant_review_volume = spark.sql(restaurant_review_volume_sql)


#Users with at least 2 restaurant reviews
min_review_per_user = 2

multi_restaurant_users_sql = f'''
        SELECT 
            u.user_id, COUNT(DISTINCT r.business_id) AS restaurant_review_count
        FROM 
            users u
        INNER JOIN 
            restaurant_reviews r
        ON u.user_id = r.user_id
        GROUP BY 
            u.user_id
        HAVING 
            COUNT(DISTINCT r.business_id) >= {min_review_per_user}
        ;
    '''

multi_restaurant_users = spark.sql(multi_restaurant_users_sql)
multi_restaurant_users.createOrReplaceTempView("multi_restaurant_users")


#Data for restaurant reviews for users with at least 2 reviews
final_reviews_sql = f'''
        SELECT 
            r.user_id, ub.user_number, ub.average_stars, ub.elite, ub.fans, ub.friends, ub.name, ub.review_count, ub.yelping_since,
            r.business_id, b.business_number, b.address, b.attributes, b.categories, b.city, b.state, b.hours, b.latitude, b.longitude, b.name, b.postal_code, b.review_count, b.stars AS business_stars,
            r.review_id, r.date, r.stars AS review_stars, r.cool, r.funny, r.useful
        FROM 
            restaurant_reviews r
        INNER JOIN 
            multi_restaurant_users u
        ON r.user_id = u.user_id
        INNER JOIN
            users ub
        ON r.user_id = ub.user_id
        INNER JOIN
            restaurants b
        ON r.business_id = b.business_id
        ;
    '''

final_reviews = spark.sql(final_reviews_sql)
final_reviews.createOrReplaceTempView("final_reviews")

In [14]:
final_reviews.select(func.col('attributes')).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|attributes                                                                                                                                                

In [15]:
final_reviews.select(func.to_json(func.col('attributes'))).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
#For each user, pull 70% for training and 30% for testing - do on user basis, so we can test user's recommendations
# Define the percentage for training and testing data
train_percentage = 0.7
test_percentage = 1 - train_percentage
seed = 42

#Random shuffle
shuffled_final = final_reviews.orderBy(func.rand(42))

# Define window specification partitioned by the user_id column and corresponding row number
window_spec = Window.partitionBy("user_id").orderBy("user_id")
final_reviews_with_row_number = shuffled_final.withColumn("row_number", func.row_number().over(window_spec))

# Calculate the total number of rows for each user
user_counts = final_reviews_with_row_number.groupBy("user_id").count()

# Calculate the number of rows to include in the training set for each user
user_test_counts = user_counts.withColumn("test_count", (func.col("count") * test_percentage).cast("int"))

#Join row number data with test count
final_with_test_counts = final_reviews_with_row_number.join(user_test_counts, "user_id", "inner")

# Create training and testing DataFrames for each user
train_data = final_with_test_counts.filter(func.col("row_number") <= (func.col("count") - func.col("test_count")))
test_data = final_with_test_counts.filter(func.col("row_number") > (func.col("count") - func.col("test_count")))

# Drop the intermediate columns
train_data = train_data.drop("row_number", "count", "test_count")
test_data = test_data.drop("row_number", "count", "test_count")

In [ ]:
test_data.count()

In [ ]:
train_data.count()

In [ ]:
#Collaborative Filtering

#Build model
#Tune maxIter, regParam, and rank
als = ALS(maxIter=5, regParam=0.01, userCol="user_number", itemCol="business_number", ratingCol="review_stars",
          coldStartStrategy="drop") 
model = als.fit(train_data)

In [ ]:
#Evaluate the model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_stars", predictionCol="prediction")
predictions = model.transform(test_data)

In [ ]:
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) =", rmse)

#Do we want to do the content-based? If we're able to process review text, could maybe do key word trends / sentiment analysis as part of it

Not sure if we have enough information on the business to make informed decision - need to unnest some of the data because all my fields are captured under attributes. If we have enough attributes, can do content-based and knn to find "similar" restaurants

Have issues with memory on my computer for the model fitting... Didn't try to do CV because it's already struggling

If we use attributes, use PCA to cut down dimensions

Try to run in Google Colab

Content-based with text analysis of reviews (TFIDF)

Content-based with attributes unnested

Schedule

Step 1: Dataset Overview
The Yelp dataset includes several types of information, but for our content-based system, we'll focus on:
Businesses: Information about individual businesses, including their categories (e.g., restaurants, bars).
Reviews: Textual reviews written by users for businesses.
Step 2: Data Preprocessing
Filter Restaurants: From the businesses data, filter out to keep only those in the 'Restaurants' category.
Aggregate Reviews: For each restaurant, aggregate all its reviews into a single text document. This creates a comprehensive text representation of each restaurant's reviews.
Text Preprocessing: Clean and preprocess the aggregated review texts for each restaurant. This involves steps like converting to lowercase, removing punctuation and stop words, and possibly stemming or lemmatization.
Step 3: Feature Extraction
TF-IDF: Apply Term Frequency-Inverse Document Frequency (TF-IDF) vectorization to the preprocessed review texts. This converts the textual data into a numerical format that captures the importance of words in relation to the document they appear in and the entire corpus of review texts. The result is a sparse matrix where each row represents a restaurant and each column represents a term's TF-IDF score.
Step 4: User Profile Creation
Identify User Preferences: For the target user, identify a set of restaurants they have rated highly or reviewed positively.
Aggregate Preferred Reviews: Aggregate the review texts of these preferred restaurants to create a "preference profile" for the user. This text represents the kind of language and themes the user appears to favor in their preferred dining experiences.
Vectorize User Profile: Apply the same TF-IDF vectorization used on the restaurant reviews to this aggregated preference profile text. This results in a TF-IDF vector representing the user's preferences.
Step 5: Recommendation Generation
Calculate Similarity: Compute the similarity between the user's preference vector and each restaurant's TF-IDF vector. This can be done using cosine similarity, which measures the cosine of the angle between two vectors, providing a similarity score between 0 and 1.
Rank Restaurants: Rank the restaurants based on their similarity scores relative to the user's preference vector. Higher scores indicate a closer match to the user's preferences as inferred from their favored reviews.
Step 6: Provide Recommendations
Select Top-N: Select the top N restaurants with the highest similarity scores. These are the system's recommendations to the user, as they are the most similar to the user's expressed preferences based on past reviews.
Example Output
Assume our target user has shown a preference for restaurants with words like "cozy", "artisanal coffee", and "homemade pastries" frequently appearing in their positive reviews. The system might recommend restaurants whose aggregated reviews also frequently mention these terms, indicating a potential match in ambiance and offerings.
Considerations
Performance: Depending on the size of the dataset, steps like TF-IDF vectorization and similarity calculation can be computationally expensive. Techniques like dimensionality reduction or efficient similarity search algorithms can help mitigate this.
Cold Start Problem: New users or restaurants without sufficient reviews pose a challenge, as there's little data to base recommendations on. Hybrid approaches or content-based features not reliant on user interaction can help address this.
Diversity and Serendipity: Pure content-based systems might over-specialize in recommending items too similar to what the user has already experienced. Incorporating elements to introduce diversity and serendipity can enhance user satisfaction.

Performance - Use recall for content-based AND collaborative filtering; we also have RMSE


Content-based filtering is a recommendation technique that utilizes information about the items themselves to make recommendations. When evaluating the performance of a content-based filtering system, you can use various metrics to assess its effectiveness. Here are some common performance metrics for content-based filtering:

Precision and Recall: Precision measures the proportion of recommended items that are relevant to the user, while recall measures the proportion of relevant items that are recommended to the user. These metrics are typically calculated at different levels, such as precision@k and recall@k, where k is the number of recommended items.

F1 Score: The F1 score is the harmonic mean of precision and recall and provides a single metric that balances both precision and recall. It's calculated as 
2
×
Precision
×
Recall
Precision
+
Recall
2× 
Precision+Recall
Precision×Recall
​
 .

Mean Average Precision (MAP): MAP measures the average precision across all users. It considers both the relevance of recommended items and their rank in the list of recommendations.

Normalized Discounted Cumulative Gain (NDCG): NDCG evaluates the ranking quality of recommended items by considering both relevance and rank. It penalizes items that are ranked lower in the list of recommendations.

Mean Reciprocal Rank (MRR): MRR measures the average rank of the first relevant item in the list of recommendations. It provides an indication of how quickly relevant items are found.

Coverage: Coverage measures the proportion of items in the catalog that are recommended to at least one user. It provides insights into the diversity of recommendations.

Novelty: Novelty measures the degree to which recommended items are different from those that the user has interacted with before. It helps ensure that recommendations introduce users to new and interesting items.

User Satisfaction: User satisfaction can be assessed through user feedback, surveys, or other qualitative measures. It provides insights into whether users find the recommendations useful and relevant.

When evaluating a content-based filtering system, it's essential to consider a combination of these metrics to gain a comprehensive understanding of its performance. Additionally, the choice of metrics may depend on the specific goals and characteristics of the recommendation system and the domain in which it operates.

Schedule
Haley to commit data to GitHub for Daemon

Due: Apr 27

Try converting to parquet file first, but it might impact nested json

Tues @ 9 PM EST - sync; have models ready with recall calculation for each model

Paper (can start Wed and finish initial draft by Fri and final edits to submit Sat); do we want to meet Fri or Sat before submitting to review or just review / comment async?
- Preprocessing
  - Text analysis
  - Train / test split
- Analyze / compare results - does one model do better at recall for certain types of restaurants than the other; what about general performance (recall)
